<h1>Box</h1>

In [2]:
# Class defining a box with load capacity
class Box:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self._items = []
        self._load = 0
        
    def can_fit(self, item):
        return self.space() >= item
    
    def space(self):
        return self.capacity - self._load
    
    def add_item(self, item):
        if not self.can_fit(item):
            raise Exception('Item of size %i does not fit.' % item)
        self._items.append(item)
        self._load += item
    
    def get_load(self):
        return self._load
    
    def __repr__(self):
        return 'Box %s (%i)' % (self._items, self.space())
    
    def __str__(self):
        return 'Box %s (%i)' % (self._items, self.space())

<b>Question</b><br>Why <code>_items</code> and <code>_load</code>?

In [4]:
box = Box(12)
box.add_item(13)
if box.can_fit(3):
    print('it fits')
else:
    print('it does not fit')

Exception: Item of size 13 does not fit.

<h1>Heuristics</h1>

In [5]:
def next_fit(items, box_size):
    
    # initialize boxes
    current_box = Box(box_size)
    boxes = [current_box]
    
    # loop items
    for item in items:
 
        if not current_box.can_fit(item):
            current_box = Box(box_size)
            boxes.append(current_box)
        current_box.add_item(item)
    
    # return list of boxes
    return boxes

<b>Challenge</b><br>There is duplicate code in the above function. How to streamline this?<br><i>Hint: what would you do first, to guarentee you can place the item in the current box?</i>

In [6]:
def next_k_fit(items, box_size, k):
    
    # initialize boxes
    boxes = []
    
    # loop items
    for item in items:
        
        # place in first box where item fits
        for box in boxes[-k:]:
            if box.can_fit(item):
                box.add_item(item)
                break
                
        # add new empty box if needed
        else:
            box = Box(box_size)
            box.add_item(item)
            boxes.append(box)
    
    # return list of boxes
    return boxes

<b>For-else</b><br>The above method can use <code>for-else</code>.

<b>Challenge</b><br>Can we get rid of <code>k_boxes</code>? How?

In [8]:
def first_fit(items, box_size):
    
    # initialize boxes
    boxes = []
    
    # loop items
    for item in items:
        
        # place in first box where item fits
        for box in boxes:
            if box.can_fit(item):
                box.add_item(item)
                break # do not check further boxes, the item was packed
        
        # add new empty box if needed
        else:
            box = Box(box_size)
            box.add_item(item)
            boxes.append(box)
            
    # return list of boxes
    return boxes

In [9]:
def best_fit(items, box_size):
    
    # initialize boxes
    boxes = []
    
    # loop items
    for item in items:
        
        # find the best box
        best_box = None
        for box in boxes:
            if box.can_fit(item) and (not best_box or box.space() < best_box.space()):
                best_box = box
        
        # add new empty box if needed
        if best_box == None:
            box = Box(box_size)
            box.add_item(item)
            boxes.append(box)
            
        # or add item to the best box
        else:
            best_box.add_item(item)
            
    # return list of boxes
    return boxes

<b>Challenge</b><br>There is a bug in the above function. Where is it?<br><i>Hint: run through the logic when placing the second item.</i>

Helper function to print boxes in a vertical list.

In [10]:
# Print solution of boxes
def print_boxes(boxes):
    space = 0
    for box in boxes:
        print(box)
        space += box.space()
    print('Total space: %i' % space)

In [15]:
#items = [5, 6, 3]
items = [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]

box_size = 9

# next_fit  next_k_fit  first_fit  best_fit
boxes = first_fit(items, box_size)
print_boxes(boxes)

boxes = next_k_fit(items, box_size, 3)
print_boxes(boxes)

Box [6, 2] (1)
Box [6, 2] (1)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [4, 2, 2] (1)
Box [3, 3] (3)
Box [7] (2)
Box [7] (2)
Box [5, 4] (0)
Box [5, 4] (0)
Box [8] (1)
Box [8] (1)
Box [5] (4)
Total space: 16
Box [6] (3)
Box [6] (3)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [4, 2, 2] (1)
Box [2, 2, 3] (2)
Box [3, 5] (1)
Box [7] (2)
Box [7] (2)
Box [5, 4] (0)
Box [8] (1)
Box [8] (1)
Box [4, 5] (0)
Total space: 16


<b>Challenge</b><br>How to implement <i>off-line</i> packing algorithms?

In [16]:
boxes = first_fit(sorted(items, reverse=True), box_size)
print_boxes(boxes)

Box [8] (1)
Box [8] (1)
Box [7, 2] (0)
Box [7, 2] (0)
Box [6, 3] (0)
Box [6, 3] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [2, 2] (5)
Total space: 7


<h1>Enum</h1>

In [18]:
from enum import Enum

# Enum of bin packing heuristics
class Heuristic(Enum):
    
    NEXT_FIT = 1
    NEXT_K_FIT = 2
    FIRST_FIT = 3
    BEST_FIT = 4
    
    # Bin pack items
    def pack(self, items, k=1):
        if self == Heuristic.NEXT_FIT:
            return next_k_fit(items, box_size, 1)
        if self == Heuristic.NEXT_K_FIT:
            return next_k_fit(items, box_size, k)
        if self == Heuristic.FIRST_FIT:
            return next_k_fit(items, box_size, float('inf'))
        if self == Heuristic.BEST_FIT:
            return best_fit(items, box_size)

<b>Challenge</b><br>Can we reduce the overall code that we need?<br><i>Hint: what heuristic(s) are special cases of other heuristic(s)?</i>

In [296]:
# values can be looped
for heuristic in Heuristic:
    print(heuristic.name)

NEXT_FIT
NEXT_K_FIT
FIRST_FIT
BEST_FIT


In [19]:
# function to print result of any heuristic it is given
def solve_and_print(heuristic, items, k=1):
    print_boxes(heuristic.pack(items, k))
    
solve_and_print(Heuristic.FIRST_FIT, items)

Box [6, 2] (1)
Box [6, 2] (1)
Box [5, 4] (0)
Box [5, 4] (0)
Box [5, 4] (0)
Box [4, 2, 2] (1)
Box [3, 3] (3)
Box [7] (2)
Box [7] (2)
Box [5, 4] (0)
Box [5, 4] (0)
Box [8] (1)
Box [8] (1)
Box [5] (4)
Total space: 16
